[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/gouthamgo/FineTuning/blob/main/lessons/module5_deployment/02_mlops_monitoring.ipynb)

# 📊 MLOps & Monitoring: Keep Your Models Healthy

**Duration:** 1.5 hours  
**Level:** Advanced  
**What You'll Learn:** How to monitor, maintain, and improve production ML systems

---

## This Is What Separates Good from Great ML Engineers!

**Real talk:** Deploying a model is only 20% of the job.

The other 80% is:
- 📊 Monitoring performance
- 🔧 Fixing issues before users notice
- 📈 Continuously improving
- 🚨 Alerting when things go wrong
- 🔄 Re-training when needed

**Companies LOVE candidates who understand production ML!**

Today you'll learn the MLOps practices that make you stand out. Let's go! 🚀

## 🎯 The 5 Pillars of Production ML

### 1. **Monitoring** 📊
Track what's happening in production

### 2. **Logging** 📝
Record everything for debugging

### 3. **Alerting** 🚨
Get notified when things go wrong

### 4. **Versioning** 🔖
Track models, data, and code

### 5. **Continuous Training** 🔄
Keep models fresh with new data

Let's implement each one!

In [ ]:
!pip install -q prometheus-client psutil pandas numpy

## 📊 Pillar 1: Comprehensive Monitoring

You need to track 4 types of metrics:

1. **Model Performance** - Is accuracy dropping?
2. **System Performance** - Is it slow? Crashing?
3. **Data Quality** - Is input data changing?
4. **Business Metrics** - Is it helping the business?

Let's build a complete monitoring system:

In [ ]:
from prometheus_client import Counter, Histogram, Gauge, Summary
import time
import numpy as np
from collections import deque

class MLMonitor:
    """Comprehensive ML model monitoring system"""
    
    def __init__(self, model_name="ml_model"):
        self.model_name = model_name
        
        # ===== MODEL PERFORMANCE METRICS =====
        
        # Total predictions made
        self.prediction_counter = Counter(
            f'{model_name}_predictions_total',
            'Total number of predictions',
            ['model_version', 'prediction_class']
        )
        
        # Prediction confidence distribution
        self.confidence_histogram = Histogram(
            f'{model_name}_prediction_confidence',
            'Distribution of prediction confidence scores',
            buckets=[0.5, 0.6, 0.7, 0.8, 0.9, 0.95, 0.99, 1.0]
        )
        
        # Current accuracy (rolling window)
        self.accuracy_gauge = Gauge(
            f'{model_name}_accuracy',
            'Current model accuracy'
        )
        
        # ===== SYSTEM PERFORMANCE METRICS =====
        
        # Latency tracking
        self.latency_histogram = Histogram(
            f'{model_name}_latency_seconds',
            'Model inference latency',
            buckets=[0.01, 0.05, 0.1, 0.5, 1.0, 2.0, 5.0]
        )
        
        # Error tracking
        self.error_counter = Counter(
            f'{model_name}_errors_total',
            'Total number of errors',
            ['error_type']
        )
        
        # Memory usage
        self.memory_gauge = Gauge(
            f'{model_name}_memory_mb',
            'Memory usage in MB'
        )
        
        # ===== DATA QUALITY METRICS =====
        
        # Input distribution (detect drift)
        self.input_length_summary = Summary(
            f'{model_name}_input_length',
            'Input text length distribution'
        )
        
        # Out-of-vocabulary rate
        self.oov_gauge = Gauge(
            f'{model_name}_oov_rate',
            'Out-of-vocabulary word rate'
        )
        
        # ===== BUSINESS METRICS =====
        
        # Escalation rate (for chatbots)
        self.escalation_gauge = Gauge(
            f'{model_name}_escalation_rate',
            'Percentage of predictions escalated to humans'
        )
        
        # Internal state for calculations
        self.recent_predictions = deque(maxlen=1000)  # Last 1000 predictions
        self.recent_confidences = deque(maxlen=1000)
        self.recent_escalations = deque(maxlen=1000)
    
    def track_prediction(self, prediction, confidence, true_label=None, 
                        model_version="v1.0", input_length=None, escalated=False):
        """Track a single prediction with all relevant metrics"""
        
        # Model performance
        self.prediction_counter.labels(
            model_version=model_version,
            prediction_class=str(prediction)
        ).inc()
        
        self.confidence_histogram.observe(confidence)
        self.recent_confidences.append(confidence)
        
        # Track accuracy if we have true label
        if true_label is not None:
            is_correct = (prediction == true_label)
            self.recent_predictions.append(is_correct)
            
            # Update rolling accuracy
            if len(self.recent_predictions) > 0:
                accuracy = sum(self.recent_predictions) / len(self.recent_predictions)
                self.accuracy_gauge.set(accuracy)
        
        # Data quality
        if input_length:
            self.input_length_summary.observe(input_length)
        
        # Business metrics
        self.recent_escalations.append(1 if escalated else 0)
        if len(self.recent_escalations) > 0:
            escalation_rate = sum(self.recent_escalations) / len(self.recent_escalations)
            self.escalation_gauge.set(escalation_rate)
    
    def track_latency(self, latency_seconds):
        """Track inference latency"""
        self.latency_histogram.observe(latency_seconds)
    
    def track_error(self, error_type):
        """Track an error"""
        self.error_counter.labels(error_type=error_type).inc()
    
    def get_summary(self):
        """Get current monitoring summary"""
        summary = {
            'total_predictions': len(self.recent_predictions),
            'current_accuracy': sum(self.recent_predictions) / len(self.recent_predictions) if self.recent_predictions else 0,
            'avg_confidence': sum(self.recent_confidences) / len(self.recent_confidences) if self.recent_confidences else 0,
            'escalation_rate': sum(self.recent_escalations) / len(self.recent_escalations) if self.recent_escalations else 0,
        }
        return summary

# Example usage
print("📊 ML Monitoring System\n" + "="*60)

monitor = MLMonitor(model_name="support_bot")

# Simulate predictions
print("\nSimulating 100 predictions...\n")
for i in range(100):
    # Simulate prediction
    prediction = np.random.choice([0, 1])
    true_label = prediction if np.random.random() > 0.15 else 1 - prediction  # 85% accuracy
    confidence = np.random.uniform(0.6, 0.99)
    escalated = confidence < 0.75
    
    # Track it
    start = time.time()
    monitor.track_prediction(
        prediction=prediction,
        confidence=confidence,
        true_label=true_label,
        input_length=np.random.randint(10, 200),
        escalated=escalated
    )
    latency = time.time() - start
    monitor.track_latency(latency)

# Get summary
summary = monitor.get_summary()
print("📈 MONITORING SUMMARY:")
print(f"  Total Predictions: {summary['total_predictions']}")
print(f"  Current Accuracy: {summary['current_accuracy']:.1%}")
print(f"  Average Confidence: {summary['avg_confidence']:.1%}")
print(f"  Escalation Rate: {summary['escalation_rate']:.1%}")

print("\n✅ This data would be visible in Prometheus/Grafana dashboards!")

## 📝 Pillar 2: Structured Logging

Logs are your time machine - they let you see exactly what happened.

**Good logging practices:**

In [ ]:
import logging
import json
from datetime import datetime

class MLLogger:
    """Structured logging for ML systems"""
    
    def __init__(self, model_name, log_level=logging.INFO):
        self.model_name = model_name
        self.logger = logging.getLogger(model_name)
        self.logger.setLevel(log_level)
        
        # Console handler
        console_handler = logging.StreamHandler()
        console_handler.setLevel(log_level)
        
        # JSON formatter for structured logging
        class JSONFormatter(logging.Formatter):
            def format(self, record):
                log_data = {
                    'timestamp': datetime.utcnow().isoformat(),
                    'level': record.levelname,
                    'model': model_name,
                    'message': record.getMessage(),
                }
                
                # Add extra fields if present
                if hasattr(record, 'extra_data'):
                    log_data.update(record.extra_data)
                
                return json.dumps(log_data)
        
        console_handler.setFormatter(JSONFormatter())
        self.logger.addHandler(console_handler)
    
    def log_prediction(self, request_id, input_text, prediction, confidence, latency_ms):
        """Log a prediction with all context"""
        extra_data = {
            'request_id': request_id,
            'input_length': len(input_text),
            'prediction': prediction,
            'confidence': confidence,
            'latency_ms': latency_ms,
        }
        
        # Create log record with extra data
        record = self.logger.makeRecord(
            self.logger.name,
            logging.INFO,
            "", 0, "Prediction made", (), None
        )
        record.extra_data = extra_data
        self.logger.handle(record)
    
    def log_error(self, request_id, error_type, error_message, stack_trace=None):
        """Log an error with context"""
        extra_data = {
            'request_id': request_id,
            'error_type': error_type,
            'error_message': str(error_message),
        }
        
        if stack_trace:
            extra_data['stack_trace'] = stack_trace
        
        record = self.logger.makeRecord(
            self.logger.name,
            logging.ERROR,
            "", 0, "Error occurred", (), None
        )
        record.extra_data = extra_data
        self.logger.handle(record)

# Example usage
print("📝 Structured Logging Example\n" + "="*60 + "\n")

logger = MLLogger("support_bot")

# Log a prediction
logger.log_prediction(
    request_id="req_123",
    input_text="How do I reset my password?",
    prediction="account_help",
    confidence=0.92,
    latency_ms=145
)

# Log an error
logger.log_error(
    request_id="req_124",
    error_type="ModelError",
    error_message="Model inference failed"
)

print("\n✅ Logs are structured (JSON) for easy parsing and analysis!")
print("   These can be sent to CloudWatch, DataDog, or Elasticsearch")

## 🚨 Pillar 3: Smart Alerting

Don't wait for users to complain - detect problems automatically!

In [ ]:
from collections import deque
from datetime import datetime, timedelta

class AlertingSystem:
    """Intelligent alerting for ML systems"""
    
    def __init__(self):
        self.alerts = []
        self.metrics_history = {
            'accuracy': deque(maxlen=1000),
            'latency': deque(maxlen=1000),
            'confidence': deque(maxlen=1000),
            'error_rate': deque(maxlen=1000),
        }
    
    def check_alerts(self, current_metrics, thresholds):
        """Check if any metrics exceed thresholds"""
        alerts = []
        
        # Accuracy dropped
        if current_metrics['accuracy'] < thresholds['min_accuracy']:
            alerts.append({
                'severity': 'CRITICAL',
                'metric': 'accuracy',
                'current': current_metrics['accuracy'],
                'threshold': thresholds['min_accuracy'],
                'message': f"Accuracy dropped to {current_metrics['accuracy']:.1%} (threshold: {thresholds['min_accuracy']:.1%})",
                'action': 'Investigate data drift or model degradation'
            })
        
        # Latency too high
        if current_metrics['latency_p95'] > thresholds['max_latency_p95']:
            alerts.append({
                'severity': 'WARNING',
                'metric': 'latency',
                'current': current_metrics['latency_p95'],
                'threshold': thresholds['max_latency_p95'],
                'message': f"P95 latency is {current_metrics['latency_p95']}ms (threshold: {thresholds['max_latency_p95']}ms)",
                'action': 'Scale up resources or optimize model'
            })
        
        # Confidence dropping (potential data drift)
        if current_metrics['avg_confidence'] < thresholds['min_confidence']:
            alerts.append({
                'severity': 'WARNING',
                'metric': 'confidence',
                'current': current_metrics['avg_confidence'],
                'threshold': thresholds['min_confidence'],
                'message': f"Average confidence dropped to {current_metrics['avg_confidence']:.1%} (threshold: {thresholds['min_confidence']:.1%})",
                'action': 'Check for distribution shift or new input patterns'
            })
        
        # Error rate too high
        if current_metrics['error_rate'] > thresholds['max_error_rate']:
            alerts.append({
                'severity': 'CRITICAL',
                'metric': 'error_rate',
                'current': current_metrics['error_rate'],
                'threshold': thresholds['max_error_rate'],
                'message': f"Error rate at {current_metrics['error_rate']:.1%} (threshold: {thresholds['max_error_rate']:.1%})",
                'action': 'Check logs immediately - service may be failing'
            })
        
        return alerts
    
    def send_alerts(self, alerts):
        """Send alerts (in production, this would email/Slack/PagerDuty)"""
        for alert in alerts:
            severity_emoji = '🚨' if alert['severity'] == 'CRITICAL' else '⚠️'
            print(f"\n{severity_emoji} {alert['severity']} ALERT")
            print(f"   Metric: {alert['metric']}")
            print(f"   {alert['message']}")
            print(f"   Recommended Action: {alert['action']}")

# Example usage
print("🚨 Alerting System Example\n" + "="*60)

alerting = AlertingSystem()

# Define thresholds
thresholds = {
    'min_accuracy': 0.80,
    'max_latency_p95': 500,  # ms
    'min_confidence': 0.70,
    'max_error_rate': 0.05,  # 5%
}

# Simulate degraded performance
print("\n📉 Simulating degraded system performance...\n")

current_metrics = {
    'accuracy': 0.75,  # Below threshold!
    'latency_p95': 650,  # Too high!
    'avg_confidence': 0.65,  # Too low!
    'error_rate': 0.02,  # OK
}

alerts = alerting.check_alerts(current_metrics, thresholds)
alerting.send_alerts(alerts)

print("\n" + "="*60)
print("\n✅ In production, these alerts would trigger:")
print("   • Slack/Email notifications")
print("   • PagerDuty incidents")
print("   • Automatic rollback to previous model version")
print("   • Runbook links for debugging")

## 🔖 Pillar 4: Model Versioning

You MUST track:
- Which model version is in production?
- What data was it trained on?
- What were the hyperparameters?
- When was it deployed?

**Best practice: Use MLflow or similar**

In [ ]:
import json
from datetime import datetime

class ModelRegistry:
    """Simple model registry for tracking versions"""
    
    def __init__(self):
        self.models = {}
    
    def register_model(self, version, metadata):
        """Register a new model version"""
        self.models[version] = {
            **metadata,
            'registered_at': datetime.now().isoformat(),
            'status': 'registered'
        }
        print(f"✅ Registered model version {version}")
    
    def promote_to_production(self, version):
        """Promote a model to production"""
        if version not in self.models:
            raise ValueError(f"Version {version} not found")
        
        # Demote current production model
        for v, data in self.models.items():
            if data['status'] == 'production':
                data['status'] = 'archived'
                data['archived_at'] = datetime.now().isoformat()
        
        # Promote new model
        self.models[version]['status'] = 'production'
        self.models[version]['deployed_at'] = datetime.now().isoformat()
        
        print(f"🚀 Promoted version {version} to production")
    
    def get_production_model(self):
        """Get currently deployed model"""
        for version, data in self.models.items():
            if data['status'] == 'production':
                return version, data
        return None, None
    
    def list_models(self):
        """List all registered models"""
        return self.models

# Example usage
print("🔖 Model Registry Example\n" + "="*60 + "\n")

registry = ModelRegistry()

# Register version 1.0
registry.register_model(
    version="1.0.0",
    metadata={
        'model_type': 'DistilBERT',
        'training_data': 'customer_qa_v1.csv',
        'num_examples': 10000,
        'accuracy': 0.87,
        'hyperparameters': {
            'learning_rate': 3e-5,
            'batch_size': 16,
            'epochs': 3,
        }
    }
)

# Deploy to production
registry.promote_to_production("1.0.0")

print("\n" + "-"*60 + "\n")

# Register version 1.1 with improvements
registry.register_model(
    version="1.1.0",
    metadata={
        'model_type': 'DistilBERT',
        'training_data': 'customer_qa_v2.csv',
        'num_examples': 15000,  # More data
        'accuracy': 0.91,  # Better!
        'hyperparameters': {
            'learning_rate': 2e-5,
            'batch_size': 32,
            'epochs': 5,
        }
    }
)

# Check production model
prod_version, prod_data = registry.get_production_model()
print(f"\n📊 Current Production Model: v{prod_version}")
print(f"   Accuracy: {prod_data['accuracy']:.1%}")
print(f"   Deployed: {prod_data['deployed_at']}")

print("\n💡 Benefits of model registry:")
print("   • Easy rollback if new model fails")
print("   • Track what changed between versions")
print("   • Reproducible experiments")
print("   • Compliance and auditing")

## 🔄 Pillar 5: Continuous Training Pipeline

Models degrade over time. You need a pipeline to:
1. Detect when retraining is needed
2. Automatically retrain
3. Evaluate new model
4. Deploy if better

**Simple CI/CD for ML:**

In [ ]:
class ContinuousTrainingPipeline:
    """Automated model retraining pipeline"""
    
    def __init__(self, monitor, registry):
        self.monitor = monitor
        self.registry = registry
    
    def should_retrain(self, current_metrics, thresholds):
        """Decide if model needs retraining"""
        reasons = []
        
        # Accuracy dropped significantly
        if current_metrics['accuracy'] < thresholds['min_accuracy']:
            reasons.append(f"Accuracy below threshold: {current_metrics['accuracy']:.1%} < {thresholds['min_accuracy']:.1%}")
        
        # Confidence dropping (data drift)
        if current_metrics['avg_confidence'] < thresholds['min_confidence']:
            reasons.append(f"Confidence dropping: {current_metrics['avg_confidence']:.1%}")
        
        # Enough new data accumulated
        if current_metrics.get('new_examples', 0) > thresholds['retrain_data_threshold']:
            reasons.append(f"New data available: {current_metrics['new_examples']} examples")
        
        # Time-based (retrain every N days)
        prod_version, prod_data = self.registry.get_production_model()
        if prod_data:
            deployed_at = datetime.fromisoformat(prod_data['deployed_at'])
            days_since_deploy = (datetime.now() - deployed_at).days
            if days_since_deploy > thresholds['max_days_since_retrain']:
                reasons.append(f"Model is {days_since_deploy} days old")
        
        return len(reasons) > 0, reasons
    
    def retrain_model(self):
        """Retrain model (simplified)"""
        print("🔄 Starting retraining...")
        print("   1. Fetching latest data...")
        print("   2. Preprocessing...")
        print("   3. Training model...")
        print("   4. Evaluating on test set...")
        
        # Simulate training
        new_accuracy = np.random.uniform(0.88, 0.93)
        
        return {
            'accuracy': new_accuracy,
            'model_path': f'/models/model_{datetime.now().strftime("%Y%m%d")}',
        }
    
    def run_pipeline(self, current_metrics, thresholds):
        """Run the full continuous training pipeline"""
        print("🔄 CONTINUOUS TRAINING PIPELINE\n" + "="*60 + "\n")
        
        # Step 1: Check if retraining needed
        should_retrain, reasons = self.should_retrain(current_metrics, thresholds)
        
        if not should_retrain:
            print("✅ Model performance is good. No retraining needed.")
            return
        
        print("⚠️ Retraining triggered!")
        print("\nReasons:")
        for reason in reasons:
            print(f"   • {reason}")
        
        print("\n" + "-"*60 + "\n")
        
        # Step 2: Retrain
        new_model = self.retrain_model()
        
        print("\n" + "-"*60 + "\n")
        
        # Step 3: Evaluate
        print("📊 Model Comparison:")
        prod_version, prod_data = self.registry.get_production_model()
        print(f"   Current (v{prod_version}): {prod_data['accuracy']:.1%}")
        print(f"   New Model: {new_model['accuracy']:.1%}")
        
        # Step 4: Deploy if better
        if new_model['accuracy'] > prod_data['accuracy']:
            new_version = self._increment_version(prod_version)
            
            self.registry.register_model(
                version=new_version,
                metadata={
                    'model_type': 'DistilBERT',
                    'accuracy': new_model['accuracy'],
                    'retrained': True,
                    'previous_version': prod_version,
                }
            )
            
            self.registry.promote_to_production(new_version)
            
            print(f"\n✅ Deployed new model v{new_version}!")
            print(f"   Improvement: {new_model['accuracy'] - prod_data['accuracy']:.2%}")
        else:
            print("\n⚠️ New model not better. Keeping current version.")
    
    def _increment_version(self, version):
        """Increment semantic version"""
        parts = version.split('.')
        parts[-1] = str(int(parts[-1]) + 1)
        return '.'.join(parts)

# Example usage
pipeline = ContinuousTrainingPipeline(monitor, registry)

# Simulate degraded metrics that trigger retraining
degraded_metrics = {
    'accuracy': 0.78,  # Dropped!
    'avg_confidence': 0.68,
    'new_examples': 5000,
}

retrain_thresholds = {
    'min_accuracy': 0.80,
    'min_confidence': 0.70,
    'retrain_data_threshold': 3000,
    'max_days_since_retrain': 30,
}

pipeline.run_pipeline(degraded_metrics, retrain_thresholds)

## 📚 Complete MLOps Checklist

Use this checklist for every production ML system:

### Pre-Deployment ✅
- [ ] Model versioning set up
- [ ] Training data versioned
- [ ] Hyperparameters documented
- [ ] Evaluation metrics defined
- [ ] Performance baseline established

### Deployment ✅
- [ ] Health check endpoint
- [ ] Metrics endpoint
- [ ] Structured logging configured
- [ ] Error tracking enabled
- [ ] Load testing completed
- [ ] Rollback plan documented

### Post-Deployment ✅
- [ ] Dashboards created (Grafana/DataDog)
- [ ] Alerts configured (PagerDuty/Slack)
- [ ] On-call rotation set up
- [ ] Incident runbooks written
- [ ] Performance reviewed weekly
- [ ] Retraining schedule defined

### Monitoring Metrics ✅
- [ ] Accuracy/F1 score
- [ ] Latency (p50, p95, p99)
- [ ] Throughput (requests/sec)
- [ ] Error rate
- [ ] Confidence distribution
- [ ] Input data distribution
- [ ] Resource usage (CPU/memory)
- [ ] Cost per prediction

## 🎯 Interview Answers: MLOps Questions

### Q: "How do you monitor models in production?"

**Your Answer:**
> "I monitor four categories of metrics:
>
> 1. **Model Performance**: Accuracy, F1, confidence scores using rolling windows to detect degradation
> 2. **System Performance**: Latency (p95/p99), throughput, error rates using Prometheus
> 3. **Data Quality**: Input distributions, OOV rates to detect drift
> 4. **Business Metrics**: Escalation rate, cost per prediction, user satisfaction
>
> I use Prometheus for metrics collection, Grafana for dashboards, and set up alerts in PagerDuty when accuracy drops below 80% or latency exceeds 500ms. In my support bot project, this caught a 15% accuracy drop within 2 hours due to new product features not in training data."

---

### Q: "How do you handle model drift?"

**Your Answer:**
> "I use a multi-stage approach:
>
> 1. **Detection**: Monitor confidence scores, input distributions, and per-class accuracy. If avg confidence drops 10% or input distribution shifts significantly (KL divergence), I investigate.
>
> 2. **Validation**: Compare recent performance (last 7 days) to baseline. If accuracy drops below threshold, trigger retraining.
>
> 3. **Retraining**: Automated pipeline collects new labeled data, retrains model, evaluates on holdout set.
>
> 4. **Deployment**: If new model is 2%+ better, deploy via blue-green deployment. Monitor for 24 hours before full rollout.
>
> I also schedule monthly retraining regardless of drift to incorporate new examples."

---

### Q: "Describe your ML deployment pipeline"

**Your Answer:**
> "My pipeline has 6 stages:
>
> 1. **Training**: Train model, track with MLflow (hyperparams, metrics, artifacts)
> 2. **Validation**: Test on holdout set, check for regression vs current prod model
> 3. **Staging**: Deploy to staging environment, run integration tests
> 4. **Canary**: Route 5% traffic to new model, monitor for 2 hours
> 5. **Gradual Rollout**: If metrics good, increase to 25%, 50%, 100% over 24 hours
> 6. **Monitor**: Track performance, ready to rollback if issues
>
> Everything is automated via GitHub Actions and Docker. Rollback takes <2 minutes using previous container version."

---

**These answers show you actually understand production ML!** 🚀

## 🎉 You're Now an MLOps Pro!

You learned:
- ✅ Comprehensive monitoring (4 types of metrics)
- ✅ Structured logging for debugging
- ✅ Smart alerting systems
- ✅ Model versioning and registry
- ✅ Continuous training pipelines

**This knowledge makes you stand out!**

90% of ML engineers can train models.  
Only 10% can maintain them in production.

You're now in that 10%! 💪

---

**Put on your resume:**
- "Implemented comprehensive monitoring system tracking 15+ production metrics"
- "Built automated retraining pipeline reducing model deployment time by 75%"
- "Set up alerting system detecting model degradation within 2 hours"

**Now go build production ML systems! You've got this! 🚀**